In [1]:
import sys
sys.path.append('/afs/ee.cooper.edu/user/t/a/tam8/documents/ml_misc/semi_supervised/')
import coreg
reload(coreg)
import trireg
reload(trireg)
sys.path.append('/afs/ee.cooper.edu/user/t/a/tam8/documents/ml_misc/ordinal/')
import simple
reload(simple)
sys.path.append('/afs/ee.cooper.edu/user/t/a/tam8/documents/ml_misc/ensemble/')
import stacking
reload(stacking)

sys.path.append('/afs/ee.cooper.edu/user/t/a/tam8/documents/ml_misc/neighbors/')
import rpfnn
reload(rpfnn)
import ann
reload(ann)

import pandas as pd
import numpy as np
from sklearn import preprocessing
import xgboost as xgb

import transformers as tforms
reload(tforms)
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion, make_union

import metrics
reload(metrics)
from sklearn.cross_validation import StratifiedKFold, train_test_split

from sklearn.base import clone

from sklearn.svm import SVC, LinearSVC, SVR
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression, LogisticRegression, ElasticNet, Ridge, Lasso, SGDRegressor
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier, DecisionTreeClassifier, DecisionTreeRegressor

from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor

from sklearn.lda import LDA
from sklearn.qda import QDA

from sklearn.neighbors import KNeighborsRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.isotonic import IsotonicRegression

from sklearn.preprocessing import StandardScaler

from collections import Counter
import minirank as mr

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from time import time

import cPickle as pickle

def wgmean(x, w):
    return np.exp(np.sum(w*np.log(x), axis=1) / np.sum(w, axis=1))

In [48]:
# Load Data
train_pd  = pd.read_pickle('saved/train_pd_enc.p')
test_pd  = pd.read_pickle('saved/test_pd_enc.p')
labels = pd.read_pickle('saved/labels.p')
test_ind = pickle.load(open('saved/test_ind.p'))
y_binned = np.load('saved/y_binned.npy')

fi = np.load('saved/feature_importances.npy')
y_binned[y_binned==6] = 5

drop_cols = ['T1_V10', 'T1_V13', 'T2_V7', 'T2_V10']
# drop_cols = train_pd.columns[fi < 0.01]


for col in drop_cols:
    train_pd.drop(col, axis=1, inplace=True)
    test_pd.drop(col, axis=1, inplace=True)


In [2]:
# Load Data le instead
# train_pd  = pd.read_pickle('saved/train_pd_l_enc.p')
# test_pd  = pd.read_pickle('saved/test_pd_l_enc.p')

train_pd  = pd.read_pickle('saved/train_pd_le_and_oh_enc.p')
test_pd  = pd.read_pickle('saved/test_pd_le_and_oh_enc.p')

labels = pd.read_pickle('saved/labels.p')
test_ind = pickle.load(open('saved/test_ind.p'))
y_binned = np.load('saved/y_binned.npy')
fi = np.load('saved/feature_importances.npy')

y_binned[y_binned==6] = 5

drop_cols = ['T1_V10', 'T1_V13', 'T2_V7', 'T2_V10']
# drop_cols = []

# drop_cols = train_pd.columns[fi < 0.01]


for col in drop_cols:
    train_pd.drop(col, axis=1, inplace=True)
    test_pd.drop(col, axis=1, inplace=True)

In [2]:
# Load Data binary instead
train_pd  = pd.read_pickle('saved/train_pd_binary_enc.p')
test_pd  = pd.read_pickle('saved/test_pd_binary_enc.p')
labels = pd.read_pickle('saved/labels.p')
test_ind = pickle.load(open('saved/test_ind.p'))
y_binned = np.load('saved/y_binned.npy')

y_binned[y_binned==6] = 5

In [3]:
X_tsne2_26 = np.load('saved/X_tsne2_26important.npy')
X_tsne2_26_train = X_tsne2_26[:len(train_pd), :]
X_tsne2_26_test = X_tsne2_26[-len(test_pd):, :]

X_tsne2 = np.load('saved/X_tsne2.npy')
X_tsne2_train = X_tsne2[:len(train_pd), :]
X_tsne2_test = X_tsne2[-len(test_pd):, :]

print X_tsne2_train.shape
print X_tsne2_test.shape
print X_tsne2_26_train.shape
print X_tsne2_26_test.shape

(50999, 2)
(51000, 2)
(50999, 2)
(51000, 2)


In [4]:
from sklearn.datasets import dump_svmlight_file, load_svmlight_file
mapped_train, _y = load_svmlight_file('saved/mapped3000_train.libsvm')

X_3000mean = mapped_train.todense()

In [5]:
train = np.array(train_pd)
test = np.array(test_pd)

X_train = train.astype(float)
X_test = test.astype(float)
y_train = np.array(labels)

try:
    X_train = np.c_[X_train, X_tsne2_26_train]
    X_test = np.c_[X_test, X_tsne2_26_test]
#     X_train = X_tsne2_26_train
#     X_test = X_tsne2_26_test
except:
    pass

try:
    X_train = np.c_[X_train, X_3000mean]
except:
    pass

holdout = False
if holdout:
    X_train, X_hold, \
    y_train, y_hold, \
    y_binned, y_binned_hold \
    = train_test_split(
        X_train, y_train, y_binned, 
        test_size=0.2, random_state=0)

"""
pipe_x = make_pipeline(
    make_union(
        tforms.IdentityTformer(),
#         make_pipeline(AddTformer(1), BoxCoxTformer()),
#         AnscombeTformer(),
    ),
#     StandardScaler(),
)
pipe_y = make_pipeline(
    tforms.IdentityTformer(),
#     tforms.BoxCoxTformer(),
#     tforms.LogTformer(),
#     tforms.AnscombeTformer(),
#     tforms.FreemanTukeyTformer(),
#     tforms.ArcsinhTformer(),
#     StandardScaler(),
    
)
pipe_x.fit(np.r_[X_train, X_test])
pipe_y.fit(y_train)

X_train = pipe_x.transform(X_train)
X_test = pipe_x.transform(X_test)
try:
    X_hold = pipe_x.transform(X_hold)
except:
    pass
# y_train = pipe_y.fit_transform(y_train)

"""

# small_n = 5000
# X_train = X_train[:small_n,:]
# y_train = y_train[:small_n]
# y_binned = y_binned[:small_n]

print 'y_train', y_train.shape
print 'X_train', X_train.shape
print 'X_test', X_test.shape
try:
    print 'X_hold', X_hold.shape
except:
    pass
print len(np.unique(y_train))
print len(np.unique(y_binned))
print type(X_train[0][0])

y_train (50999,)
X_train (50999, 3125)
X_test (51000, 125)
X_hold 50
5
<class 'numpy.matrixlib.defmatrix.matrix'>


In [ ]:
from sklearn.base import clone
%pdb off

gbm = xgb.XGBRegressor(
    objective="reg:linear",
    n_estimators=600,
    learning_rate=0.005,
#     gamma=0.0,
    max_depth=9,
    min_child_weight=6,
#     max_delta_step=10,
    subsample=0.7,
    colsample_bytree=0.7,
#     scale_pos_weight=1.0,
    base_score=0.5,
    nthread=7,
    seed=np.random.randint(0,100),
    silent=True,
)

meta_gbm = xgb.XGBRegressor(
    objective="reg:linear",
    n_estimators=322,
    learning_rate=0.01,
#     gamma=0.0,
    max_depth=8,
    min_child_weight=5,
#     max_delta_step=10,
    subsample=0.6,
    colsample_bytree=1.0,
#     scale_pos_weight=1.0,
    base_score=0.5,
    nthread=7,
    seed=np.random.randint(0,100),
    silent=True,
)

gbm_bin = xgb.XGBClassifier(
    objective="binary:logistic",
    n_estimators=500,
    learning_rate=0.01,
#     gamma=0.0,
    max_depth=8,
    min_child_weight=5,
#     max_delta_step=10,
    subsample=0.8,
    colsample_bytree=0.8,
#     scale_pos_weight=1.0,
    base_score=0.5,
    nthread=8,
    seed=np.random.randint(0,100),
    silent=True,
)


ord_gbm_bin = simple.SimpleOrdinalClassifier(gbm_bin, n_jobs=1)
ord_gaussnb_bin = simple.SimpleOrdinalClassifier(GaussianNB(), n_jobs=-1)
ord_logistic_bin = simple.SimpleOrdinalClassifier(LogisticRegression(), n_jobs=-1)
ord_rf_bin = simple.SimpleOrdinalClassifier(RandomForestClassifier(n_estimators=200, n_jobs=-1), n_jobs=1)
ord_ada_bin = simple.SimpleOrdinalClassifier(AdaBoostClassifier(n_estimators=200), n_jobs=-1)
ord_dt = simple.SimpleOrdinalClassifier(DecisionTreeClassifier(), n_jobs=-1)
ord_lda = simple.SimpleOrdinalClassifier(LDA(solver='lsqr'), n_jobs=-1)

# clf0 = simple.SimpleOrdinalClassifier(base_estimator_type=GaussianNB, base_estimator_params={}, n_jobs=1)
# clf0 = simple.SimpleOrdinalClassifier(SVC(probability=True), n_jobs=8)
# clf0 = simple.SimpleOrdinalClassifier(clf_nn, n_jobs=1)
# clf0 = simple.SimpleOrdinalClassifier(base_estimator_type=KerasClassifier,
#                                       base_estimator_params=nn_params,
#                                       label_transformer=label_bin_tform,
#                                       n_jobs=1)

stack = stacking.Stacking([
        ord_gaussnb_bin,
        ord_lda,
#         ord_rf_bin,
#         ord_ada_bin,
#         ord_logistic_bin,
        ord_gbm_bin,
        ann.ann(no_trees=50, num_neighbors=100, weights='distance'),
#         KNeighborsRegressor(n_neighbors=20),

        LinearRegression(),
        Ridge(), 
#         BaggingRegressor(n_estimators=400, max_features=0.8, n_jobs=-1),
#         ExtraTreesRegressor(n_estimators=400, n_jobs=-1), 
#         gbm,
    ],
#                           meta_gbm,
#                           LinearRegression(),
                          gbm,
                          fit_params={
        'base0_y': 'y_binned',
        'base1_y': 'y_binned',
        'base2_y': 'y_binned',
    },
                          pred_params={
#         'base0_weighted':True,
#         'base1_weighted':True,
    },
                          include_orig_feats=True,
                          use_probs=True,
                         )

# clfs = [clf0, gbm] 
# clfs = [clone(gbm), stack]
# clfs = [LinearRegression(), stack]
# clfs = [stack]
# clfs = [BaggingRegressor(n_estimators=200, max_features=0.8, n_jobs=-1)]

# clfs = [stack]
clfs = [LinearRegression()]

# clfs = [rpfnn.rpfnn(leaf_size=100, no_trees=10, num_neighbors=100)]
# clfs = [ann.ann(no_trees=50, num_neighbors=100, weights='distance')]

# clfs = [ord_gaussnb_bin]
# clfs = [simple.SimpleOrdinalClassifier(LDA(solver='lsqr'), n_jobs=-1)]
# clfs = [clf1, clf2, ]


# h = [clone(clf1), clone(clf2)]
# clf = coreg.CoReg(h=h, T=5, verbose=True, n_jobs=1)

scores = []
scores_base = []
lols = []
n_reps = 1
k = 3
for reps in range(n_reps):
    skf = StratifiedKFold(y_binned, n_folds=k,
                          shuffle=True,
                          random_state=np.random.randint(0,100))
    for ii, (train, valid) in enumerate(skf):
        
        
#         history = LossHistory()   # for keras
        print 'Fold %d' % ii,
        X_train_k = X_train[train]
        X_valid_k = X_train[valid]
        y_train_k = y_train[train]
        y_valid_k = y_train[valid]
        y_train_binned_k = y_binned[train]
        y_valid_binned_k = y_binned[valid]     
        
        
        X_all = np.r_[X_train_k, X_valid_k]
        y_all = np.r_[y_train_k, np.nan*np.ones(len(y_valid_k))]
        
        tic = time()
#         clf.fit(X_train_k, y_train_k)
#         clf.fit(X_all, y_all)


        stack.extra_data['y_binned'] = y_train_binned_k
        for clf in clfs:
            clf.fit(X_train_k, pipe_y.transform(y_train_k))
#             clf.fit(X_train_k, y_train_binned_k)
#             clf.fit(X_train_k, y_train_binned_k,
#                    nb_epoch=60, batch_size=1024*16, )
#             clf.fit(X_train_k, y_train_k, 
#                     base0_y=y_train_binned_k,
#                     base1_y=y_train_binned_k,
# #                     base2_y=y_train_binned_k,
#                    )
    
    
        
        # Minirank
#         w, theta = mr.ordinal_logistic_fit(X_train_k, y_train_k, verbose=False,
#                                 solver='TNC')

        
        toc = time() - tic
        print 'Train time: %2.3f s\t' % toc,
        tic = time()

        
        
        valid_preds = [clf.predict_weighted(X_valid_k,) 
                       if hasattr(clf, 'predict_weighted') 
                       else clf.predict(X_valid_k)
                       for clf in clfs]
        
#         valid_preds = [clf.predict_weighted(X_valid_k, batch_size=1024*16) 
#                if hasattr(clf, 'predict_weighted') 
#                else clf.predict(X_valid_k)
#                for clf in clfs]
        
        try:
            lol = metrics.normalized_gini(y_valid_k, 
                                          clfs[0].predict_weighted(X_valid_k, geometric=True))
            lols.append(lol)
        except:
            pass

        # Minirank
#         valid_preds = mr.ordinal_logistic_predict(w, theta, X_valid_k)
        
        valid_base_preds = np.mean([clf.predict(X_valid_k) for clf in clfs
                                   ], 
                                   axis=0)

        
        score = [metrics.normalized_gini(y_valid_k, v) for v in valid_preds]
        score_base = metrics.normalized_gini(y_valid_k, valid_base_preds)
        
        toc_pred = time() - tic
        print 'Pred time: %2.3f s\t' % toc_pred, 
        
        print 'Score', score
        print 'Score avg ensemble', score_base
        scores.append(score)
        scores_base.append(score_base)
        
#         break
        
        
print "done"
print 'Score:', np.array(scores).mean(axis=0)
print 'Base Score:', np.array(scores_base).mean()
print 'lol', lols, np.mean(lols)

In [28]:
clf

rpfnn(leaf_size=None, no_trees=None, num_neighbors=10)

In [93]:
print y_train_k.shape
print y_train_binned_k.shape
# gbm is ~ 0.381  @ 30 sec per fold?
# gbm on binned labels ~ 0.378 @ 30 sec per fold... ~_~
# ordinal nn binned is like 0.34 with stdscaling X 60 epochs

# the stack is worse?????? 0.379

(33998,)
(33998,)


In [83]:

print 'Score:', np.array(scores).mean(axis=0)
print 'Base Score:', np.array(scores_base).mean()
print 'lol', lols, np.mean(lols)

Score: [ 0.37856345]
Base Score: 0.378563452073
lol [] nan


In [25]:
metrics.normalized_gini(y_valid_k, clfs[0].predict_weighted(X_valid_k, geometric=True))

0.37712908899661984

In [64]:
print y_pred_base_hold.shape
print y_hold[:, None].shape

(10200, 3)
(10200, 1)


In [72]:
##### OK, just to check early stopping on XGB when used as a meta

gbm = xgb.XGBRegressor(
    objective="reg:linear",
    n_estimators=600,
    learning_rate=0.005,
#     gamma=0.0,
    max_depth=9,
    min_child_weight=6,
#     max_delta_step=10,
    subsample=0.7,
    colsample_bytree=0.7,
#     scale_pos_weight=1.0,
    base_score=0.5,
    nthread=7,
    seed=np.random.randint(0,100),
    silent=True,
)

gbm2 = xgb.XGBRegressor(
    objective="reg:linear",
    n_estimators=100,
    learning_rate=0.01,
#     gamma=0.0,
    max_depth=8,
    min_child_weight=5,
#     max_delta_step=10,
    subsample=0.8,
    colsample_bytree=1.0,
#     scale_pos_weight=1.0,
    base_score=0.5,
    nthread=7,
    seed=np.random.randint(0,100),
    silent=True,
)

gbm_bin = xgb.XGBClassifier(
    objective="binary:logistic",
    n_estimators=500,
    learning_rate=0.01,
#     gamma=0.0,
    max_depth=8,
    min_child_weight=5,
#     max_delta_step=10,
    subsample=0.8,
    colsample_bytree=0.8,
#     scale_pos_weight=1.0,
    base_score=0.5,
    nthread=8,
    seed=np.random.randint(0,100),
    silent=True,
)

meta_params = {}
meta_params["objective"] = "reg:linear"
meta_params["eta"] = 0.01
meta_params["min_child_weight"] = 5
meta_params["subsample"] = 0.6
meta_params["colsample_bytree"] = 1.0
# meta_params["max_delta_step"] = 10
meta_params["silent"] = 1
meta_params["max_depth"] = 8 #7
meta_params["scale_pos_weight"] = 1.0
# meta_params["gamma"] = 0

meta_plst = list(meta_params.items())

max_rounds = 1000



ord_gbm_bin = simple.SimpleOrdinalClassifier(gbm_bin, n_jobs=1)
ord_gaussnb_bin = simple.SimpleOrdinalClassifier(GaussianNB(), n_jobs=1)

stack = stacking.Stacking([
        ord_gaussnb_bin,
#         KNeighborsRegressor(n_neighbors=20),
#         ord_gbm_bin,
        LinearRegression(),
        Ridge(), 
#         ExtraTreesRegressor(n_estimators=400, n_jobs=-1), 
#         gbm,
    ],
#                           LinearRegression(),
                          gbm2,
                          pred_params={
#         'base0_weighted':True,
#         'base1_weighted':True,
                                      }
                         )

# clfs = [clf0, gbm] 
# clfs = [clone(gbm), stack]
# clfs = [LinearRegression(), stack]
clfs = [ord_gaussnb_bin, LinearRegression(), Ridge(),
       ExtraTreesRegressor(n_estimators=400, n_jobs=-1), 
       gbm]
# clfs = [ord_gaussnb_bin]
# clfs = [clf1, clf2, ]


# h = [clone(clf1), clone(clf2)]
# clf = coreg.CoReg(h=h, T=5, verbose=True, n_jobs=1)

scores = []
scores_base = []
lols = []
n_reps = 1
k = 3
for reps in range(n_reps):
    skf = StratifiedKFold(y_train, n_folds=k,
                          shuffle=True,
                          random_state=np.random.randint(0,100))
    for ii, (train, valid) in enumerate(skf):
        
        
#         history = LossHistory()   # for keras
        print 'Fold %d' % ii,
        X_train_k = X_train[train]
        X_valid_k = X_train[valid]
        y_train_k = y_train[train]
        y_valid_k = y_train[valid]
        y_train_binned_k = y_binned[train]
        y_valid_binned_k = y_binned[valid]     
        
        
        X_all = np.r_[X_train_k, X_valid_k]
        y_all = np.r_[y_train_k, np.nan*np.ones(len(y_valid_k))]
        
        tic = time()
#         clf.fit(X_train_k, y_train_k)
#         clf.fit(X_all, y_all)
        
        for ii, clf in enumerate(clfs):
            if ii > 0:
                clf.fit(X_train_k, y_train_k)
            else:
                clf.fit(X_train_k, y_train_binned_k)
    
        y_pred_base_valk = np.array([clf.predict(X_valid_k) for clf in clfs]).T
        y_pred_base_hold = np.array([clf.predict(X_hold) for clf in clfs]).T
    
    
            #create a train and validation dmatrices 
        xgtrain = xgb.DMatrix(y_pred_base_valk, label=y_valid_k)
        xgval = xgb.DMatrix(y_pred_base_hold, label=y_hold)

        #train using early stopping and predict
        watchlist = [(xgtrain, 'train'),(xgval, 'val')]
        model = xgb.train(meta_plst, xgtrain, 
                          max_rounds, watchlist, 
                          early_stopping_rounds=20)
        # preds1 = model.predict(xgtest)

        
        
        
        
        
        # Minirank
#         w, theta = mr.ordinal_logistic_fit(X_train_k, y_train_k, verbose=False,
#                                 solver='TNC')

        
        toc = time() - tic
        print 'Train time: %2.3f s\t' % toc,
        tic = time()

        
        
        valid_preds = [clf.predict_weighted(X_valid_k,) 
                       if hasattr(clf, 'predict_weighted') 
                       else clf.predict(X_valid_k)
                       for clf in clfs]
        
#         valid_preds = [clf.predict_weighted(X_valid_k, batch_size=1024*16) 
#                if hasattr(clf, 'predict_weighted') 
#                else clf.predict(X_valid_k)
#                for clf in clfs]
        
        try:
            lol = metrics.normalized_gini(y_valid_k, 
                                          clfs[0].predict_weighted(X_valid_k, geometric=True))
            lols.append(lol)
        except:
            pass

        # Minirank
#         valid_preds = mr.ordinal_logistic_predict(w, theta, X_valid_k)
        
        valid_base_preds = np.mean([clf.predict(X_valid_k) for clf in clfs
                                   ], 
                                   axis=0)

        
        score = [metrics.normalized_gini(y_valid_k, v) for v in valid_preds]
        score_base = metrics.normalized_gini(y_valid_k, valid_base_preds)
        
        toc_pred = time() - tic
        print 'Pred time: %2.3f s\t' % toc_pred, 
        
        print 'Score', score
        print 'Score avg ensemble', score_base
        scores.append(score)
        scores_base.append(score_base)
        
#         break
        
        
print "done"
print 'Score:', np.array(scores).mean(axis=0)
print 'Base Score:', np.array(scores_base).mean()
print 'lol', lols, np.mean(lols)

Will train until val error hasn't decreased in 20 rounds.
[0]	train-rmse:5.329849	val-rmse:5.395958
[1]	train-rmse:5.302631	val-rmse:5.370376
[2]	train-rmse:5.276110	val-rmse:5.345001
[3]	train-rmse:5.249720	val-rmse:5.320238
[4]	train-rmse:5.223888	val-rmse:5.295659
[5]	train-rmse:5.198101	val-rmse:5.271391
[6]	train-rmse:5.172885	val-rmse:5.247104
[7]	train-rmse:5.148187	val-rmse:5.223596
[8]	train-rmse:5.123522	val-rmse:5.200033
[9]	train-rmse:5.100122	val-rmse:5.178236
[10]	train-rmse:5.076361	val-rmse:5.155858
[11]	train-rmse:5.053448	val-rmse:5.133976
[12]	train-rmse:5.030273	val-rmse:5.112043
[13]	train-rmse:5.007358	val-rmse:5.090184
[14]	train-rmse:4.984417	val-rmse:5.068751
[15]	train-rmse:4.962422	val-rmse:5.048325
[16]	train-rmse:4.940381	val-rmse:5.028071
[17]	train-rmse:4.918781	val-rmse:5.008236
[18]	train-rmse:4.897426	val-rmse:4.987994
[19]	train-rmse:4.876969	val-rmse:4.969168
[20]	train-rmse:4.856151	val-rmse:4.949934
[21]	train-rmse:4.836106	val-rmse:4.931154
[22]	t

Fold 0 Train time: 170.482 s	Pred time: 3.649 s	

[322]	train-rmse:3.458762	val-rmse:3.874237
[323]	train-rmse:3.458232	val-rmse:3.874173
[324]	train-rmse:3.458042	val-rmse:3.874223
[325]	train-rmse:3.457505	val-rmse:3.874361
[326]	train-rmse:3.457053	val-rmse:3.874321
[327]	train-rmse:3.456802	val-rmse:3.874245
Stopping. Best iteration:
[307]	train-rmse:3.467036	val-rmse:3.873316

Will train until val error hasn't decreased in 20 rounds.
[0]	train-rmse:5.306577	val-rmse:5.395870
[1]	train-rmse:5.279452	val-rmse:5.369708
[2]	train-rmse:5.252771	val-rmse:5.344047
[3]	train-rmse:5.226677	val-rmse:5.318620
[4]	train-rmse:5.200452	val-rmse:5.293792
[5]	train-rmse:5.174356	val-rmse:5.269023
[6]	train-rmse:5.149210	val-rmse:5.245500
[7]	train-rmse:5.124567	val-rmse:5.221835
[8]	train-rmse:5.100512	val-rmse:5.198489
[9]	train-rmse:5.076005	val-rmse:5.174926
[10]	train-rmse:5.051366	val-rmse:5.151367
[11]	train-rmse:5.027265	val-rmse:5.128829
[12]	train-rmse:5.003814	val-rmse:5.106824
[13]	train-rmse:4.981085	val-rmse:5.085626
[14]	train-rmse

Score [0.020934110217732695, 0.32113828425423985, 0.3229898231550764, 0.297693788250269, 0.36360149143208276]
Score avg ensemble 0.348236119642
Fold 1 Train time: 178.504 s	Pred time: 3.691 s	

Stopping. Best iteration:
[276]	train-rmse:3.434055	val-rmse:3.869041

Will train until val error hasn't decreased in 20 rounds.
[0]	train-rmse:5.260703	val-rmse:5.396377
[1]	train-rmse:5.233791	val-rmse:5.370673
[2]	train-rmse:5.207413	val-rmse:5.345889
[3]	train-rmse:5.180664	val-rmse:5.320631
[4]	train-rmse:5.154742	val-rmse:5.296618
[5]	train-rmse:5.128829	val-rmse:5.272711
[6]	train-rmse:5.102811	val-rmse:5.248287
[7]	train-rmse:5.077386	val-rmse:5.224842
[8]	train-rmse:5.053463	val-rmse:5.202157
[9]	train-rmse:5.029206	val-rmse:5.179982
[10]	train-rmse:5.005175	val-rmse:5.157667
[11]	train-rmse:4.981343	val-rmse:5.135851
[12]	train-rmse:4.958085	val-rmse:5.114446
[13]	train-rmse:4.934888	val-rmse:5.093126
[14]	train-rmse:4.912112	val-rmse:5.072383
[15]	train-rmse:4.889708	val-rmse:5.051811
[16]	train-rmse:4.867610	val-rmse:5.031323
[17]	train-rmse:4.845726	val-rmse:5.011089
[18]	train-rmse:4.824983	val-rmse:4.991868
[19]	train-rmse:4.804113	val-rmse:4.972331
[20]	train-rmse:4.783

Score [0.08474084973769001, 0.3253787197571052, 0.32614693393780525, 0.31089631014905533, 0.36996104832264304]
Score avg ensemble 0.352724181624
Fold 2 Train time: 171.064 s	Pred time: 3.705 s	Score [-0.0047924121230007045, 0.3310942900211449, 0.33415621113905364, 0.313292922927283, 0.38320644001786747]
Score avg ensemble 0.355084154036
done
Score: [ 0.03362752  0.32587043  0.32776432  0.30729434  0.37225633]
Base Score: 0.352014818434
lol [0.037577726126945814, 0.0883661633689758, -0.004185386943965162] 0.0405861675173


[340]	train-rmse:3.358703	val-rmse:3.887532
[341]	train-rmse:3.358339	val-rmse:3.887462
[342]	train-rmse:3.356925	val-rmse:3.887619
[343]	train-rmse:3.355405	val-rmse:3.887876
[344]	train-rmse:3.355127	val-rmse:3.887876
[345]	train-rmse:3.354469	val-rmse:3.887878
[346]	train-rmse:3.353774	val-rmse:3.888135
[347]	train-rmse:3.352992	val-rmse:3.888116
Stopping. Best iteration:
[327]	train-rmse:3.367603	val-rmse:3.886618



In [13]:
params = {}
params["objective"] = "reg:linear"
params["eta"] = 0.005
params["min_child_weight"] = 6
params["subsample"] = 0.7
params["colsample_bytree"] = 0.7
# params["max_delta_step"] = 10
params["silent"] = 1
params["max_depth"] = 9 #7
params["scale_pos_weight"] = 1.0
# params["gamma"] = 0


plst = list(params.items())

max_rounds = 2000
# xgtest = xgb.DMatrix(X_test)


skf = StratifiedKFold(y_binned, n_folds=3,
                      shuffle=True,
                      random_state=np.random.randint(0,100))
train, valid= iter(skf).next()
X_train_k = X_train[train]
X_valid_k = X_train[valid]
y_train_k = y_train[train]
y_valid_k = y_train[valid]


#create a train and validation dmatrices 
xgtrain = xgb.DMatrix(X_train_k, label=y_train_k)
xgval = xgb.DMatrix(X_valid_k, label=y_valid_k)

#train using early stopping and predict
watchlist = [(xgtrain, 'train'),(xgval, 'val')]
model = xgb.train(plst, xgtrain, max_rounds, watchlist, early_stopping_rounds=120)

preds_val = model.predict(xgval)

s = metrics.normalized_gini(y_valid_k, preds_val)
print s

Will train until val error hasn't decreased in 120 rounds.
[0]	train-rmse:5.337576	val-rmse:5.322946
[1]	train-rmse:5.323978	val-rmse:5.310519
[2]	train-rmse:5.310580	val-rmse:5.298182
[3]	train-rmse:5.296911	val-rmse:5.285621
[4]	train-rmse:5.283404	val-rmse:5.273452
[5]	train-rmse:5.270011	val-rmse:5.261607
[6]	train-rmse:5.256827	val-rmse:5.249586
[7]	train-rmse:5.243795	val-rmse:5.237566
[8]	train-rmse:5.230467	val-rmse:5.225552
[9]	train-rmse:5.217557	val-rmse:5.213836
[10]	train-rmse:5.204528	val-rmse:5.202345
[11]	train-rmse:5.191781	val-rmse:5.190744
[12]	train-rmse:5.178873	val-rmse:5.179070
[13]	train-rmse:5.166155	val-rmse:5.167631
[14]	train-rmse:5.153434	val-rmse:5.155980
[15]	train-rmse:5.141041	val-rmse:5.144683
[16]	train-rmse:5.128650	val-rmse:5.133505
[17]	train-rmse:5.116412	val-rmse:5.122573
[18]	train-rmse:5.104103	val-rmse:5.111464
[19]	train-rmse:5.091579	val-rmse:5.100379
[20]	train-rmse:5.079823	val-rmse:5.089691
[21]	train-rmse:5.067692	val-rmse:5.078730
[22]	

0.353874800827


[1173]	train-rmse:3.122358	val-rmse:3.811956
Stopping. Best iteration:
[1053]	train-rmse:3.154454	val-rmse:3.811431



In [11]:
model.best_iteration

522

# Forest Neighbors

In [67]:
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint as sp_randint

reload(rpfnn)
clf = rpfnn.rpfnn(leaf_size=50, no_trees=10, num_neighbors=10)

param_dist = {"leaf_size": sp_randint(20, 500),
              "no_trees": sp_randint(10, 400),
              "num_neighbors": sp_randint(1, 200),
             }

n_iter_search = 20
random_search = RandomizedSearchCV(clf, 
                                   param_distributions=param_dist,
                                   n_iter=n_iter_search,
                                   cv=skf,
                                   scoring=metrics.gini_score,
                                   n_jobs=-1)

In [68]:
random_search.fit(X_train, y_train)


RandomizedSearchCV(cv=sklearn.cross_validation.StratifiedKFold(labels=[1 3 ..., 5 4], n_folds=3, shuffle=True, random_state=14),
          error_score='raise',
          estimator=rpfnn(leaf_size=50, no_trees=10, num_neighbors=10),
          fit_params={}, iid=True, n_iter=20, n_jobs=-1,
          param_distributions={'no_trees': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f5c59929a10>, 'leaf_size': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f5c59929790>, 'num_neighbors': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f5c59929a90>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring=make_scorer(gini), verbose=0)

In [73]:
print random_search.best_score_
print random_search.best_params_

0.297775058583
{'no_trees': 301, 'leaf_size': 77, 'num_neighbors': 115}


# Pred

In [16]:
tic = time()
stack.extra_data['y_binned'] = y_binned
stack.fit(X_train, y_train)
toc = time() - tic

print toc

604.879694939


# Generate submission

In [21]:
preds = stack.predict(X_test)

In [22]:
preds_pd = pd.DataFrame({"Id": test_ind, "Hazard": preds})
preds_pd = preds_pd.set_index('Id')
preds_pd.to_csv('submissions/stack_random_crap.csv')